In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.callbacks import TensorBoard
import time 


In [ ]:
DATADIR = "E://tf_capstone//Combined//training"
CATEGORIES = ["red", "yellow", "green"]

#data is agumented but not balanced (use weights for unbalanced data)

cd E://tf_capstone//Combined

/training
    /red
        /red1.png
    /yellow
        /yellow1.png
    /green
        /green1.png
        
        
/testing
    /red
        /red1.png
    /yellow
        /yellow1.png
    /green
        /green1.png

In [ ]:
for category in CATEGORIES:
    path=os.path.join(DATADIR, category) #path for subdirectories
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
        #plt.imshow(img_array)
        #plt.show()
        break
    break

In [ ]:
IMG_SIZE = 120 #change the image size (best alt at 120)
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array)
plt.show()

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try: 
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)#UNCHANGED #GRAY_SCALE #COLOR #COLOR_2 #ANYCOLOR
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
create_training_data()

import random
random.shuffle(training_data)

X= []
y= []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X= np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE,3)
X= X/255.0

In [ ]:
from keras.models import load_model
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

            model.fit(X, y, batch_size=32, epochs=2, validation_split=0.25)
            
            model.save('my_model.h5')